In [1]:
# Importing libraries
import pandas as pd
import re
import category_encoders as ce
import numpy as np
from sklearn.preprocessing import StandardScaler

In [2]:
# Importing as df
df = pd.read_csv('master.csv')
df.head()

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers


In [3]:
# Droping columns with high numbers of missing values
df.drop(columns=['HDI for year'], inplace=True)
df[' gdp_for_year ($) '] = df[' gdp_for_year ($) '].apply(lambda x: int(re.sub(r'[, ]', '', x)))
df.dtypes

country                object
year                    int64
sex                    object
age                    object
suicides_no             int64
population              int64
suicides/100k pop     float64
country-year           object
 gdp_for_year ($)       int64
gdp_per_capita ($)      int64
generation             object
dtype: object

In [4]:
# Calculating modal values for categorical variables
modalValues = df.groupby(['country', 'year', 'age', 'sex', 'generation'])['population'].sum().reset_index()
modalValues = modalValues.groupby(['country', 'year']).\
    apply(lambda x: x.loc[x['population'].idxmax()]).reset_index(drop=True).\
    drop(columns=['population'])
modalValues.head()

,country,year,age,sex,generation
0,Albania,1987,5-14 years,male,Generation X
1,Albania,1988,5-14 years,male,Generation X
2,Albania,1989,5-14 years,male,Generation X
3,Albania,1992,5-14 years,male,Millenials
4,Albania,1993,5-14 years,male,Millenials


In [5]:
# Totalizing values
totalValues = df.groupby(['country', 'year']).\
    agg({'suicides_no': 'sum',
         'population': 'sum',
         'gdp_per_capita ($)': 'mean',
        ' gdp_for_year ($) ': 'mean'}).reset_index()
totalValues[' gdp_for_year ($) '] = totalValues[' gdp_for_year ($) '].astype(int)
totalValues.head()

,country,year,suicides_no,population,gdp_per_capita ($),gdp_for_year ($)
0,Albania,1987,73,2709600,796.0,2156624900
1,Albania,1988,63,2764300,769.0,2126000000
2,Albania,1989,68,2803100,833.0,2335124988
3,Albania,1992,47,2822500,251.0,709452584
4,Albania,1993,73,2807300,437.0,1228071038


In [6]:
# Merging dataframes and creating response variable
dfPro = totalValues.merge(modalValues, on=['country', 'year'], how='left').\
    sort_values(['country', 'year'])
dfPro['suicideRate'] = dfPro['suicides_no']*100000/dfPro['population']
dfPro.head()

,country,year,suicides_no,population,gdp_per_capita ($),gdp_for_year ($),age,sex,generation,suicideRate
0,Albania,1987,73,2709600,796.0,2156624900,5-14 years,male,Generation X,2.694125
1,Albania,1988,63,2764300,769.0,2126000000,5-14 years,male,Generation X,2.279058
2,Albania,1989,68,2803100,833.0,2335124988,5-14 years,male,Generation X,2.425886
3,Albania,1992,47,2822500,251.0,709452584,5-14 years,male,Millenials,1.665190
4,Albania,1993,73,2807300,437.0,1228071038,5-14 years,male,Millenials,2.600363


In [7]:
# Selecting countries with information between 2009-2013
selCountries = dfPro[(dfPro['year'] >= 2009) & (dfPro['year'] <= 2013)].\
    groupby('country')['year'].nunique().reset_index().query('year==5')
selCountries.shape

(75, 2)

In [8]:
# Filtering dataframe
dfPro = dfPro[(dfPro['year'] >= 2009) & (dfPro['year'] <= 2013) & (dfPro['country'].isin(selCountries['country']))]
dfPro.shape

(375, 10)

In [9]:
# Number of unique countries
totalCountries = len(dfPro['country'].unique())
totalCountries

75

In [10]:
# Encoding categorical variables
mapping = [{'col': 'age','mapping':{'5-14 years': 0,
                                    '15-24 years': 1,
                                    '35-54 years': 2,
                                    '55-74 years': 3}},
           {'col': 'generation', 'mapping':{'Millenials': 0,
                                            'Generation Z': 1,
                                            'Generation X': 2,
                                            'Boomers': 3,
                                            'Silent': 4}},
          {'col': 'sex', 'mapping': {'male': 0,
                                     'female': 1}}]

encoder = ce.OrdinalEncoder(cols = 'age',
                           return_df=True,
                           mapping=mapping)

encoder.fit(dfPro)

OrdinalEncoder(cols=['age'],
               mapping=[{'col': 'age', 'data_type': dtype('O'),
                         'mapping': 5-14 years     0
15-24 years    1
35-54 years    2
55-74 years    3
dtype: int64},
                        {'col': 'generation', 'data_type': dtype('O'),
                         'mapping': Millenials      0
Generation Z    1
Generation X    2
Boomers         3
Silent          4
dtype: int64},
                        {'col': 'sex', 'data_type': dtype('O'),
                         'mapping': male      0
female    1
dtype: int64}])

In [11]:
# Train & test division by country preparation
countryID = pd.DataFrame({'country': dfPro['country'].drop_duplicates(),
                          'id': np.arange(0, totalCountries)})
dfPro = dfPro.merge(countryID, on=['country'], how='left')

In [12]:
# Train and test size
trainSize = int(totalCountries*0.7)
testSize = totalCountries - trainSize
print(trainSize, testSize)

52 23


In [13]:
# Creating random samples
dataSample = np.arange(0, totalCountries)
trainSample = np.random.choice(totalCountries, size=trainSize, replace=False)
testSample = [i for i in dataSample if i not in trainSample]

In [14]:
# Dividing
TrainDF = dfPro[dfPro['id'].isin(list(trainSample))].drop(columns='id')
TestDF = dfPro[dfPro['id'].isin(list(testSample))].drop(columns='id')

In [15]:
# Encoding train ant test
TrainDF = encoder.transform(TrainDF)
TestDF = encoder.transform(TestDF)
TrainDF.head(10)

,country,year,suicides_no,population,gdp_per_capita ($),gdp_for_year ($),age,sex,generation,suicideRate
0,Argentina,2009,2884,37158001,8961.0,332976484578,2,1,3,7.761451
1,Argentina,2010,2943,37578454,11273.0,423627422092,2,1,2,7.831615
2,Argentina,2011,2912,38015739,13946.0,530163281575,2,1,2,7.659985
3,Argentina,2012,3248,38441778,14203.0,545982375701,2,1,2,8.449141
4,Argentina,2013,2987,38859125,14206.0,552025140252,2,1,2,7.686740
10,Australia,2009,2337,20529317,45128.0,926448240318,2,1,3,11.383720
11,Australia,2010,2420,20847547,54887.0,1144260547873,2,1,2,11.608080
12,Australia,2011,2392,20881910,66770.0,1394280784778,2,1,2,11.454891
13,Australia,2012,2580,21238909,72669.0,1543411012580,2,1,2,12.147517
14,Australia,2013,2608,21600180,72856.0,1573696522007,2,1,2,12.073973


Given that suicide rate is the variable to predict and it is a linear relationship between suicides_no and population, they will not be considered, only their lagged values.

In [16]:
# Rolling window function
def create_dataset(data, columns):
    auxDf1 = data.reset_index().copy()
    auxDf2 = auxDf1.copy()
    auxDf2['index'] = auxDf2['index'] + 1
    rezDF = pd.merge(auxDf1.drop(columns=columns),
                     auxDf2[columns + ['index', 'country']], on=['index', 'country'],
                    how='inner')
    return rezDF

In [17]:
# Rolling window
TrainDF = create_dataset(TrainDF, ['population', 'suicides_no'])
TestDF = create_dataset(TestDF, ['population', 'suicides_no'])

In [18]:
# renaming columns & Dividing x & Y matrix
TrainDF = TrainDF.rename(columns={'population': 'population-1', 'suicides_no': 'suicides_no-1'}).\
               drop(columns=['index']).sort_values(['year', 'country']).reset_index(drop=True)
TestDF = TestDF.rename(columns={'population': 'population-1', 'suicides_no': 'suicides_no-1'}).\
              drop(columns=['index']).sort_values(['year', 'country']).reset_index(drop=True)
TrainDF.head()

,country,year,gdp_per_capita ($),gdp_for_year ($),age,sex,generation,suicideRate,population-1,suicides_no-1
0,Argentina,2010,11273.0,423627422092,2,1,2,7.831615,37158001,2884
1,Australia,2010,54887.0,1144260547873,2,1,2,11.608080,20529317,2337
2,Austria,2010,49181.0,391892746545,2,0,2,15.862616,7946894,1278
3,Bahamas,2010,30239.0,10095760000,2,1,2,2.995187,327813,6
4,Bahrain,2010,22572.0,25713271277,2,0,2,1.667896,1092922,36


Longitudinal data is a stack of multiple cross sectional datasets, so makes sense we need to standardize the variables for each year individually.

In [23]:
class LongitudinalScaler:
    def __init__(self, train, test):
        self.TrainData = train.copy()
        self.TestData = test.copy()
        self.scalerDict = {}
        self.ScaledTrain = pd.DataFrame()
        self.ScaledTest = pd.DataFrame()
    
    def createScaler(self, years, colDrop):
        for year in years:
            aux = self.TrainData.copy()
            aux = aux[aux['year'] == year].drop(columns=colDrop)
            colNames = aux.columns
            scalerName = 'scaler' + str(year)
            scaler = StandardScaler()
            aux = scaler.fit_transform(aux)
            aux = pd.DataFrame(aux, columns=colNames)
            self.scalerDict[scalerName] = scaler
            self.ScaledTrain = pd.concat([self.ScaledTrain, aux], axis=0, ignore_index=True)
    
    def scalingTest(self, years, colDrop):
        for year in years:
            aux = self.TestData.copy()
            aux = aux[aux['year'] == year].drop(columns=colDrop)
            colNames = aux.columns
            scalerName = 'scaler' + str(year)
            scaler = self.scalerDict[scalerName]
            aux = scaler.transform(aux)
            aux = pd.DataFrame(aux, columns=colNames)
            self.ScaledTest = pd.concat([self.ScaledTest, aux], axis=0, ignore_index=True)

In [24]:
# Scaling Train
LSC = LongitudinalScaler(train=TrainDF, test=TestDF)
LSC.createScaler(years=[2010, 2011, 2012, 2013], colDrop=['country', 'year', 'suicideRate'])
LSC.scalingTest(years=[2010, 2011, 2012, 2013], colDrop=['country', 'year', 'suicideRate'])

In [25]:
# Saving Train & test scaled
TrainScaled = LSC.ScaledTrain.copy()
TestScaled = LSC.ScaledTest.copy()

In [26]:
# Adding ids
TrainScaled[['country', 'year', 'suicideRate']] = TrainDF[['country', 'year', 'suicideRate']]
TestScaled[['country', 'year', 'suicideRate']] = TestDF[['country', 'year', 'suicideRate']]
TestScaled.head()

,gdp_per_capita ($),gdp_for_year ($),age,sex,generation,population-1,suicides_no-1,country,year,suicideRate
0,-0.949239,-0.356430,0.228086,0.96225,0.126435,-0.483939,-0.451251,Armenia,2010,2.727723
1,0.843802,-0.143504,0.228086,-1.03923,0.126435,-0.337808,-0.215837,Belgium,2010,19.713704
2,-0.801940,-0.337866,0.228086,0.96225,0.126435,-0.395481,-0.353362,Bulgaria,2010,11.992617
3,-0.845607,-0.331708,0.228086,-1.03923,0.126435,-0.327364,-0.280215,Cuba,2010,14.525658
4,0.236059,-0.349111,0.228086,0.96225,0.126435,-0.521480,-0.453773,Cyprus,2010,4.827966


Now it is needed to reshape the data into a numpy array with a (Country, periods, variables) structure. This is required for RNN modeling and its going to be done in the next notebook

In [29]:
# Sorting values by country
TrainScaled.sort_values(['country', 'year'], inplace=True)
TestScaled.sort_values(['country', 'year'], inplace=True)
TrainScaled.head()

,gdp_per_capita ($),gdp_for_year ($),age,sex,generation,population-1,suicides_no-1,country,year,suicideRate
0,-0.630091,-0.170405,0.228086,0.962250,0.126435,0.189113,-0.111221,Argentina,2010,7.831615
52,-0.569353,-0.148444,0.228086,0.962250,0.200000,0.191271,-0.097785,Argentina,2011,7.659985
104,-0.555124,-0.140554,0.228086,0.856349,0.200000,0.194880,-0.098542,Argentina,2012,8.449141
156,-0.577180,-0.139636,0.247436,0.925820,0.247436,0.198003,-0.050675,Argentina,2013,7.686740
1,1.151472,0.153115,0.228086,0.962250,0.126435,-0.135590,-0.176921,Australia,2010,11.608080


In [30]:
# Exporting data
TrainScaled.to_csv('TrainScaled.csv')
TestScaled.to_csv('TestScaled.csv')